In [ ]:
import math
import uproot
from pathlib import Path
import awkward as ak
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tqdm

## Getting the file and tree

In [ ]:
# fname = Path("/Users/alexandertuna/Downloads/cms/trackingNtuple.root")
fname = Path("/Users/alexandertuna/Downloads/cms/lst_playing/data/trackingNtuple.2025_03_15_03h02m56s.root")
if not fname.exists():
    raise Exception("shit")

In [ ]:
file = uproot.open(f"{fname}")
print(file.keys())

In [ ]:
tree = uproot.open(f"{fname}:trackingNtuple/tree")
print(tree)

In [ ]:
def get_prefixes(col):
    return sorted(list(set([obj.split("_")[0] for obj in col])))
print(get_prefixes(tree.keys()))

## Getting branches into a data array

In [ ]:
print(tree.keys())

In [ ]:
data = tree.arrays([
    'trk_pt', 'trk_eta', 'trk_phi',

    'ph2_isBarrel', 'ph2_isLower', 'ph2_isUpper', 'ph2_isStack', 
    'ph2_order', 'ph2_ring', 'ph2_rod', 'ph2_detId', 
    'ph2_subdet', 'ph2_layer', 'ph2_side', 'ph2_module', 
    'ph2_moduleType', 'ph2_trkIdx', 'ph2_onTrk_x', 'ph2_onTrk_y', 
    'ph2_onTrk_z', 'ph2_onTrk_xx', 'ph2_onTrk_xy', 'ph2_onTrk_yy', 
    'ph2_onTrk_yz', 'ph2_onTrk_zz', 'ph2_onTrk_zx', 'ph2_tcandIdx', 
    'ph2_seeIdx', 'ph2_simHitIdx', 'ph2_simType', 'ph2_x', 'ph2_y', 
    'ph2_z', 'ph2_xx', 'ph2_xy', 'ph2_yy', 
    'ph2_yz', 'ph2_zz', 'ph2_zx', 'ph2_radL', 
    'ph2_bbxi', 'ph2_usedMask', 'ph2_clustSize',

    'simhit_x', 'simhit_y', 'simhit_z',
    'simhit_px', 'simhit_py', 'simhit_pz',
    'simhit_tof', 'simhit_particle', 'simhit_simTrkIdx', 

    'sim_event', 'sim_bunchCrossing', 'sim_pdgId',
    'sim_genPdgIds', 'sim_isFromBHadron', 
    'sim_px', 'sim_py', 'sim_pz', 
    'sim_pt', 'sim_eta', 'sim_phi',
    #'sim_pca_pt', 'sim_pca_eta', 'sim_pca_lambda', 'sim_pca_cotTheta', 
    #'sim_pca_phi', 'sim_pca_dxy', 'sim_pca_dz', 'sim_q', 
    #'sim_nValid', 'sim_nPixel', 'sim_nStrip', 
    #'sim_nLay', 'sim_nPixelLay', 'sim_n3DLay', 
    #'sim_nTrackerHits', 'sim_nRecoClusters', 
    #'sim_trkIdx', 'sim_trkShareFrac', 'sim_seedIdx', 
    'sim_parentVtxIdx', 'sim_decayVtxIdx', 'sim_simHitIdx',
])
data["simhit_pt"] = np.sqrt(data.simhit_px**2 + data.simhit_py**2)
data["simhit_costheta"] = 0

data["simhit_rt"] = np.sqrt(data.simhit_x**2 + data.simhit_y**2)
data["simhit_costheta"] = ((data.simhit_x * data.simhit_px) + (data.simhit_y * data.simhit_py)) / (data.simhit_pt * data.simhit_rt)
data["simhit_phi"] = np.atan2(data.simhit_y, data.simhit_x)
data["ph2_phi"] = np.atan2(data.ph2_y, data.ph2_x)
data["ph2_rt"] = np.sqrt(data.ph2_x**2 + data.ph2_y**2)

## Coordinate conversions

In [ ]:
def eta(x, y, z):
    r_perp = np.sqrt(x**2 + y**2)
    theta = np.atan2(r_perp, z)
    return -np.log(np.tan(theta / 2.0))

def phi(x, y):
    return np.atan2(y, x)

## Test plot: simhit TOF

In [ ]:
# bins = np.arange(-1, 1.02, 0.02)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.simhit_tof), bins=np.arange(-1, 35))
    # ax.hist(ak.flatten(data.simhit_tof[data.simhit_pt > 0.6]), bins=np.arange(-1, 35))
    #ax.set_xlabel("cos($\\theta$) of $p_{T, sim}$ and $x_{T, sim}$")
    #ax.set_ylabel("Hits (ph2_*)")
    #ax.set_title("Hits with associated sim track")
axs[1].semilogy();

## Convincing myself the ph2-trk matching is legit

In [ ]:
# for each event
#   for each hit
#     print the eta, phi of the hit
#     print the pt, eta, phi of matching tracks

In [ ]:
for event in range(2):

    ph2_n = len(data.ph2_x[event])
    print("")
    print(f"Event {event} has {ph2_n} hits")

    for hit in range(ph2_n):

        if hit > 20:
            break

        # ph2 eta,phi
        ph2_eta = eta(data.ph2_x[event, hit], data.ph2_y[event, hit], data.ph2_z[event, hit])
        ph2_phi = phi(data.ph2_x[event, hit], data.ph2_y[event, hit])

        # trk matching
        ph2_trk_n = len(data.ph2_trkIdx[event, hit])
        for trk in range(ph2_trk_n):
            trkIdx = data.ph2_trkIdx[event, hit, trk]
            trk_pt = data.trk_pt[event, trkIdx]
            trk_eta = data.trk_eta[event, trkIdx]
            trk_phi = data.trk_phi[event, trkIdx]
        if ph2_trk_n == 0:
            msg = ""
        elif ph2_trk_n == 1:
            msg = f": {trk_pt:4.1f}, {trk_eta:5.2f}, {trk_phi:5.2f}"
        else:
            msg = f"Whoa!!!"

        # announce
        size = data.ph2_clustSize[event][hit]
        print(f"Hit has size {size} at {ph2_eta:5.2f},{ph2_phi:5.2f} matches {ph2_trk_n} tracks {msg}")


## Creating `ph2_pt` array with track-matching logic

## Loop version

In [ ]:
ph2_pt_all = []
ph2_ntrk_all = []

nevent = len(data.ph2_x)

for event in tqdm.tqdm(range(nevent)):
    continue
    
    ph2_n = len(data.ph2_x[event])
    ph2_pt = [0]*ph2_n
    ph2_ntrk = [0]*ph2_n

    for hit in range(ph2_n):

        ph2_trk_n = len(data.ph2_trkIdx[event, hit])
        if ph2_trk_n == 0:
            continue

        pts = [data.trk_pt[event, data.ph2_trkIdx[event, hit, trk]]
               for trk in
               range(ph2_trk_n)]

        ph2_pt[hit] = max(pts)
        ph2_ntrk[hit] = ph2_trk_n

    ph2_pt_all.append( ph2_pt )
    ph2_ntrk_all.append( ph2_ntrk )

ph2_pt = ak.Array(ph2_pt_all)
ph2_ntrk = ak.Array(ph2_ntrk_all)

## Semi-vectorized version

In [ ]:
vec_ph2_ntrk = ak.num(data.ph2_trkIdx, axis=-1)

def fancy_index(data, index):
    counts = ak.num(index, axis=-1)
    flattened_idx = ak.flatten(index, axis=-1)
    out_flat = data[flattened_idx]
    out = ak.unflatten(out_flat, counts)
    return out

_pt = []
for event in tqdm.tqdm(range(len(data.ph2_x))):
    _pt.append( fancy_index(data.trk_pt[event], data.ph2_trkIdx[event]) )
vec_ph2_pts = ak.Array(_pt)
vec_ph2_pt = ak.fill_none(ak.max(vec_ph2_pts, axis=-1), 0)

#print(ph2_pt.type)
#print(vec_ph2_pt.type)

#print(ak.array_equal(ph2_ntrk, vec_ph2_ntrk))
#print(ak.array_equal(ph2_pt, vec_ph2_pt))

## Quick test: indexing

In [ ]:
pt = ak.Array([5, 6])
idx = ak.Array([[0, 0, 0, 0],
                [1, 1, 1],
                [0, 1, 0]])
_cts = ak.num(idx, axis=-1)
_flt = ak.flatten(idx)
_pts = pt[_flt]
_unf = ak.unflatten(_pts, _cts)
print(_unf)

# pt[idx]

## Another indexing test

In [ ]:
def fancy_index(data, index):
    counts = ak.num(index, axis=-1)
    counts = ak.flatten(counts)
    flattened_idx = ak.flatten(index, axis=-1)
    out_flat = data[flattened_idx]
    out = ak.unflatten(out_flat, counts, axis=-1)
    return out

print(data.sim_pt.type)
print(data.simhit_simTrkIdx.type)
print(data.ph2_simHitIdx.type)
_ = data.sim_pt[data.simhit_simTrkIdx]
_ = fancy_index(data.simhit_pt, data.ph2_simHitIdx)
# _ = data.sim_pt[data.ph2_simHitIdx]

In [ ]:
original = ak.Array([[1, 2, 3, 4], [], [5, 6, 7], [8, 9]])
ak.unflatten(original, [2, 2, 1, 2, 1, 1], axis=-1).show()

## Quick test: are simhit_p{x, y, z} branches filled?

In [ ]:
for event in range(3):
    ph2_n = len(data.ph2_x[event])
    print(f"Event {event}")
    for i_hit, hit in enumerate(range(ph2_n)):
        msg = f"Hit {i_hit:02}: "
        msg += f"ph2_x,y,z = ("
        msg += f"{data.ph2_x[event, hit]:5.1f}, "
        msg += f"{data.ph2_y[event, hit]:5.1f}, "
        msg += f"{data.ph2_z[event, hit]:5.1f}). "
        for simhit in data.ph2_simHitIdx[event, hit]:
            x, y, px, py = [
                data.simhit_x[event, simhit],
                data.simhit_y[event, simhit],
                data.simhit_px[event, simhit],
                data.simhit_py[event, simhit],
            ]
            position, momentum = np.array([x, y]), np.array([px, py])
            costheta = np.dot(position, momentum) / (np.linalg.norm(position) * np.linalg.norm(momentum))
            pt = np.sqrt(data.simhit_px[event, simhit]**2 + data.simhit_py[event, simhit]**2)
            msg += f"simhit_x,y,z = ("
            #msg += f"{data.simhit_x[event, simhit]:5.1f}, "
            #msg += f"{data.simhit_y[event, simhit]:5.1f}, "
            #msg += f"{data.simhit_z[event, simhit]:5.1f}). "
            msg += f"simhit_pt = {pt:5.1f}. "
            msg += f"simhit_costheta = {costheta:.5f}. "
            msg += f"simhit_costheta = {data.simhit_costheta[event, simhit]:.5f}. "
        # msg += f"trk_pt = {data.ph2_pt[event, hit]:5.1f}"
        print(msg)
        if hit > 10:
            break
    print("")
   

## Creating `ph2_pt` array with truth-matching logic

In [ ]:
def fancy_index(data, index):
    counts = ak.num(index, axis=-1)
    flattened_idx = ak.flatten(index, axis=-1)
    out_flat = data[flattened_idx]
    out = ak.unflatten(out_flat, counts)
    return out

_pt = []
_costheta = []
_phi = []
for event in tqdm.tqdm(range(len(data.ph2_x))):
    _pt.append( fancy_index(data.simhit_pt[event], data.ph2_simHitIdx[event]) )
    _phi.append( fancy_index(data.simhit_phi[event], data.ph2_simHitIdx[event]) )
    _costheta.append( fancy_index(data.simhit_costheta[event], data.ph2_simHitIdx[event]) )

print("Making a big array ...")
vec_ph2_pts = ak.Array(_pt)
print("Making a big array ...")
vec_ph2_phis = ak.Array(_phi)
print("Making a big array ...")
vec_ph2_costheta = ak.Array(_costheta)

print("Collapsing the simhit dim ...")
vec2_ph2_pt = ak.fill_none(ak.max(vec_ph2_pts, axis=-1), 0)
vec2_ph2_phi = ak.fill_none(ak.max(vec_ph2_phis, axis=-1), 0)
vec2_ph2_costheta = ak.fill_none(ak.max(vec_ph2_costheta, axis=-1), 0)
vec2_ph2_ntrk = ak.num(data.ph2_simHitIdx, axis=-1)

#print(ph2_pt.type)
print(vec_ph2_pt.type)
print(vec2_ph2_pt.type)

#print(ak.array_equal(ph2_ntrk, vec_ph2_ntrk))
#print(ak.array_equal(ph2_pt, vec_ph2_pt))

In [ ]:
print(vec_ph2_pt[2])
print(vec2_ph2_pt[2])

## Vectorized version

In [ ]:
def fancy_index(data, index):
    counts = ak.num(index, axis=-1)
    counts = ak.flatten(counts)
    flattened_idx = ak.flatten(index, axis=-1)
    out_flat = data[flattened_idx]
    out = ak.unflatten(out_flat, counts, axis=-1)
    return out * np.float64(1.0)

_tmp = fancy_index(data.simhit_pt, data.ph2_simHitIdx)

print(vec_ph2_pts)
print(_tmp)
print(vec_ph2_pts.type)
print(_tmp.type)
print(ak.array_equal(vec_ph2_pts, _tmp))
print(ak.almost_equal(vec_ph2_pts, _tmp))
def tuna_equal(a, b):
    return ak.flatten(a, axis=None) == ak.flatten(b, axis=None)
print(tuna_equal(vec_ph2_pts, _tmp))
xxx = tuna_equal(vec_ph2_pts, _tmp)
print(ak.sum(xxx), len(xxx))


## Assigning values for ph2_pt and ph2_ntrk

In [ ]:
data["ph2_pt"] = vec2_ph2_pt
data["ph2_simphi"] = vec2_ph2_phi
data["ph2_costheta"] = vec2_ph2_costheta
data["ph2_nsim"] = vec2_ph2_ntrk

## Making ph2_dphi

In [ ]:
def dphi(a, b):
    return np.abs(((a - b) + np.pi) % (2 * np.pi) - np.pi)
    #dab = (a - b) / (2 * np.pi)
    #mask = dab > np.pi
    #dab[mask] = 2 * np.pi - dab[mask]
    #return dab

data["ph2_dphi"] = dphi(data.ph2_phi, data.ph2_simphi)

print(data.ph2_phi)
print(data.ph2_simphi)
print(data.ph2_dphi)

## Plotting ph2_pt, ph2_costheta, ph2_dphi, and ph2_nsim

In [ ]:
bins = np.arange(-0.5, 9.5, 1)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.ph2_nsim), bins=bins)
    ax.set_xlabel("N(sim. hit)")
    ax.set_ylabel("Hits (ph2_*)")
axs[1].semilogy();

In [ ]:
bins = np.arange(-1, 1.02, 0.02)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.ph2_costheta), bins=bins)
    ax.set_xlabel("cos($\\theta$) of $p_{T, sim}$ and $x_{T, sim}$")
    ax.set_ylabel("Hits (ph2_*)")
    ax.set_title("All hits")
axs[1].semilogy();

In [ ]:
bins = np.arange(-1, 1.02, 0.02)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.ph2_costheta[data.ph2_pt > 0]), bins=bins)
    ax.set_xlabel("cos($\\phi$) of $p_{T, sim}$ and $x_{T, sim}$")
    ax.set_ylabel("Hits (ph2_*)")
    ax.set_title("Hits with associated sim track")
axs[1].semilogy();

In [ ]:
bins = np.arange(0, 3.2, 0.05)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    # ax.hist(ak.flatten(data.ph2_dphi[ph2_pt > 0.6]), bins=bins)
    ax.hist(ak.flatten(data.ph2_dphi), bins=bins)
    ax.set_xlabel("dphi(hit, sim. hit) [rad]")
    ax.set_ylabel("Hits (ph2_*)")
    # ax.set_title("Hits with associated sim track")
    ax.set_title("All hits")
axs[1].semilogy();

In [ ]:
bins = np.arange(0, 3.2, 0.05)
mask = (data.ph2_pt > 0.6)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.ph2_dphi[mask]), bins=bins)
    ax.set_xlabel("dphi(hit, sim. hit) [rad]")
    ax.set_ylabel("Hits (ph2_*)")
    ax.set_title("Hits with associated sim track")
axs[1].semilogy();

In [ ]:
bins = np.arange(-0.001, 0.25, 0.001)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
mask = (data.ph2_pt > 0.6)
for ax in axs:
    ax.hist(ak.flatten(data.ph2_rt[mask] * data.ph2_dphi[mask]), bins=bins)
    ax.set_xlabel("r * dphi(hit, sim. hit) [cm]")
    ax.set_ylabel("Hits (ph2_*)")
    ax.set_title("Hits with associated sim track")
axs[1].semilogy();

In [ ]:
bins = np.arange(-0.001, 0.03, 0.0001)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
mask = (data.ph2_pt > 0.6)
for ax in axs:
    ax.hist(ak.flatten(data.ph2_rt[mask] * data.ph2_dphi[mask]), bins=bins)
    ax.set_xlabel("r * dphi(hit, sim. hit) [cm]")
    ax.set_ylabel("Hits (ph2_*)")
    ax.set_title("Hits with associated sim track")
axs[1].semilogy();

In [ ]:
# bins = np.arange(-0.001, 0.25, 0.001)
bins = [np.arange(-0.001, 0.5, 0.005),
        np.arange(-0.5, 19.5, 1)]
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
mask = (data.ph2_pt > 0.6)
for ax in axs:
    ax.hist2d(ak.flatten(data.ph2_rt[mask] * data.ph2_dphi[mask]).to_numpy(),
              ak.flatten(data.ph2_clustSize[mask]).to_numpy(),
              norm=mpl.colors.LogNorm(),
              bins=bins,
              )
    ax.set_xlabel("r * dphi(hit, sim. hit) [cm]")
    ax.set_ylabel("Cluster size")
    ax.set_title("Hits with associated sim track")
axs[1].set_xlim([0.001, None])
axs[1].semilogx();

In [ ]:
bins = np.arange(0, 250, 5)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
axs[0].hist(ak.flatten(data.ph2_pt), bins=bins)
axs[1].hist(ak.flatten(data.ph2_pt[data.ph2_pt > 0.5]), bins=bins)
for ax in axs:
    ax.set_xlabel("$p_{T}$ [GeV]")
    ax.set_ylabel("Hits (ph2_*)")
axs[0].set_title("All hits")
axs[1].set_title("Hits matched to sim. track with $p_{T}$ > 0.5 GeV");

## Plotting ph2_clustSize

In [ ]:
mask = data.ph2_pt > 0.6
bins = np.arange(-0.5, 34.5, 1)
bin_centers = (bins[:-1] + bins[1:]) / 2.0

In [ ]:
bins = np.arange(-0.5, 34.5, 1)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.ph2_clustSize), bins=bins)
    ax.set_title("All hits")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("Hits (ph2_*)")
axs[1].semilogy();

In [ ]:
bins = np.arange(-0.5, 34.5, 1)
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
for ax in axs:
    ax.hist(ak.flatten(data.ph2_clustSize[mask]), bins=bins)
    ax.set_title("Hits matched to sim. track with $p_{T}$ > 0.6 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("Hits (ph2_*)")
#axs[1].set_ylim([1, None])
axs[1].semilogy();

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(14, 4))
bins = [np.arange(-0.5, 199.5, 4), np.arange(-0.5, 14.5)]
for it, ax in enumerate(axs):
    _, _, _, im = ax.hist2d(ak.flatten(data.ph2_pt[mask]).to_numpy(),
                            ak.flatten(data.ph2_clustSize[mask]).to_numpy(),
                            norm=(mpl.colors.LogNorm() if it==2 else None),
                            bins=bins, cmin=0.5, cmap="inferno")
    ax.set_xlabel("Sim. $p_{T}$ [GeV]")
    ax.set_ylabel("Cluster size")
axs[1].set_xlim([0.4, None])
axs[1].semilogx();

In [ ]:
bins = np.arange(-0.5, 34.5, 1)
hist, bin_edges = np.histogram(ak.flatten(data.ph2_clustSize[mask]), bins=bins, density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0

cdf = np.cumsum(hist * np.diff(bin_edges))

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)
for ax in axs:
    ax.plot(bin_centers, cdf, marker=".")
    ax.set_title("Hits matched to sim. track with $p_{T}$ > 0.6 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("CDF")
    ax.grid()
axs[1].set_ylim([0.99, 1.0]);

In [ ]:
mask_barrel = (data.ph2_pt > 0.6) & (data.ph2_order == 0)
hist, bin_edges = np.histogram(ak.flatten(data.ph2_clustSize[mask_barrel]), bins=bins, density=True)
cdf = np.cumsum(hist * np.diff(bin_edges))

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)
for ax in axs:
    ax.plot(bin_centers, cdf, marker=".")
    ax.set_title("Barrel hits where $p_{T}$ > 0.6 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("CDF")
    ax.grid()
axs[1].set_ylim([0.99, 1.0]);

In [ ]:
mask_endcap = (data.ph2_pt > 0.6) & (data.ph2_order == 1)
hist, bin_edges = np.histogram(ak.flatten(data.ph2_clustSize[mask_endcap]), bins=bins, density=True)
cdf = np.cumsum(hist * np.diff(bin_edges))

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)
for ax in axs:
    ax.plot(bin_centers, cdf, marker=".")
    ax.set_title("Endcap hits where $p_{T}$ > 0.6 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("CDF")
    ax.grid()
axs[1].set_ylim([0.99, 1.0]);

In [ ]:
mask_lowpt = (data.ph2_pt > 0.6) & (data.ph2_pt < 2)
hist, bin_edges = np.histogram(ak.flatten(data.ph2_clustSize[mask_lowpt]), bins=bins, density=True)
cdf = np.cumsum(hist * np.diff(bin_edges))

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)
for ax in axs:
    ax.plot(bin_centers, cdf, marker=".")
    ax.set_title("Hits where 0.6 < $p_{T}$ < 2 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("CDF")
    ax.grid()
axs[1].set_ylim([0.99, 1.0]);

In [ ]:
mask_medpt = (data.ph2_pt > 2) & (data.ph2_pt < 5)
hist, bin_edges = np.histogram(ak.flatten(data.ph2_clustSize[mask_medpt]), bins=bins, density=True)
cdf = np.cumsum(hist * np.diff(bin_edges))

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)
for ax in axs:
    ax.plot(bin_centers, cdf, marker=".")
    ax.set_title("Hits where 2 < $p_{T}$ < 5 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("CDF")
    ax.grid()
axs[1].set_ylim([0.99, 1.0]);

In [ ]:
mask_hipt = (data.ph2_pt > 5)
hist, bin_edges = np.histogram(ak.flatten(data.ph2_clustSize[mask_hipt]), bins=bins, density=True)
cdf = np.cumsum(hist * np.diff(bin_edges))

fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)
for ax in axs:
    ax.plot(bin_centers, cdf, marker=".")
    ax.set_title("Hits where $p_{T}$ > 5 GeV")
    ax.set_xlabel("Cluster size")
    ax.set_ylabel("CDF")
    ax.grid()
axs[1].set_ylim([0.99, 1.0]);

## Checking sim_E

In [ ]:
bins = np.arange(0, 220, 2)
fig, ax = plt.subplots()
m = 0.1057
muon = 13
# m2 = E2 - p2 -> E2 = m2 + p2
sim_E = np.sqrt(data.sim_px**2 + data.sim_py**2 + data.sim_pz**2 + m**2)
ax.hist(ak.flatten(sim_E[np.abs(data.sim_pdgId) == 13]), bins=bins)
ax.set_xlabel("Muon $E^{2}$ [GeV]")
ax.set_ylabel("Number of muons");

## Checking sim_pdgId

In [ ]:
bins = np.arange(-220, 220)
fig, ax = plt.subplots()
ax.hist(ak.flatten(data.sim_pdgId), bins=bins)
ax.set_xlabel("Sim. PDG ID")
ax.set_ylabel("Number of sim. particles")
ax.semilogy()

## Getting sim_p for each simhit

In [ ]:
print(data.simhit_pt[0].type)
print(data.simhit_simTrkIdx[0].type)
print(data.sim_pt[0].type)
print(data.sim_simHitIdx[0].type)
print(ak.min(data.simhit_simTrkIdx[0]))
print(ak.max(data.simhit_simTrkIdx[0]))
print(ak.min(data.sim_simHitIdx[0]))
print(ak.max(data.sim_simHitIdx[0]))

print(data.simhit_simTrkIdx[0])
print(data.sim_pt[0][data.simhit_simTrkIdx[0]])
print(data.simhit_pt[0])
print(data.simhit_pt[0] - data.sim_pt[0][data.simhit_simTrkIdx[0]])

In [ ]:
def fancy_index(data, index):
    counts = ak.num(index, axis=-1)
    flattened_idx = ak.flatten(index, axis=-1)
    out_flat = data[flattened_idx]
    out = ak.unflatten(out_flat, counts)
    return out

# simhit_simTrk_pt = fancy_index(data.sim_pt, data.simhit_simTrkIdx)
# print(data.sim_pt[data.simhit_simTrkIdx])
# simhit_simTrk_pt = ak.fill_none(simhit_simTrk_pt, 0)
simhit_simTrk_pt = data.sim_pt[data.simhit_simTrkIdx]

In [ ]:
print(data.simhit_simTrkIdx.type)
print(data.sim_pt[data.simhit_simTrkIdx].type)
print(simhit_simTrk_pt.type)
print(data.simhit_pt.type)

In [ ]:
bins = np.arange(-10, 150)
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
fig.subplots_adjust(wspace=0.25)

ax[0].hist(ak.flatten(simhit_simTrk_pt - data.simhit_pt), bins=bins)
ax[0].set_xlabel("simhit_simTrk_pt - simhit_pt")
ax[0].set_ylabel("simhits")
ax[0].semilogy()

ax[1].hist2d(ak.flatten(simhit_simTrk_pt).to_numpy(),
             ak.flatten(data.simhit_pt).to_numpy(),
             bins=[np.arange(-10, 210), np.arange(-10, 210)], cmin=0.5,
             )
ax[1].set_xlabel("simhit_simTrk_pt")
ax[1].set_ylabel("simhit_pt");